# Introduction

I will explore the problem in following stages:

1.  **Hypothesis Generation** – understanding the problem better by brainstorming possible factors that can impact the outcome
2.  **Data Exploration** – looking at categorical and continuous feature summaries and making inferences about the data.
3.  **Data Cleaning** – imputing missing values in the data and checking for outliers
4.  **Feature Engineering** – modifying existing variables and creating new ones for analysis
5.  **Model Building** – making predictive models on the data


## 1. Hypothesis Generation

This involves understanding the problem and making some hypothesis about what could potentially have a good impact on the outcome. This is done BEFORE looking at the data, and we end up creating a laundry list of the different analysis which we can potentially perform if data is available.

### The Problem Statement

Understanding the problem statement is the first and foremost step:

> In this competition, you will forecast the demand of a product for a given week, at a particular store. The dataset you are given consists of 9 weeks of sales transactions in Mexico. Every week, there are delivery trucks that deliver products to the vendors. Each transaction consists of sales and returns. Returns are the products that are unsold and expired. The demand for a product in a certain week is defined as the sales this week subtracted by the return next week.

So the idea is to find out the demand of a product (sales - returns) per client, and store which impacts the sales of a product. Let’s think about some of the analysis that can be done and come up with certain hypothesis.

### The Hypotheses

I came up with the following hypothesis while thinking about the problem. Since we’re talking about stores and products, lets make different sets for each.

**Store/Client Level Hypotheses:**

1.  **Town type:** Stores located in urban or Tier 1 towns should have higher sales because of the higher income levels of people there.
2.  **Population Density:** Stores located in densely populated areas should have higher sales because of more demand.
3.  **Store Capacity:** Stores which are very big in size should have higher sales as they act like one-stop-shops and people would prefer getting everything from one place
4.  **Competitors:** Stores having similar establishments nearby should have less sales because of more competition.
5.  **Marketing:** Stores which have a good marketing division should have higher sales as it will be able to attract customers through the right offers and advertising.
6.  **Location:** Stores located within popular marketplaces should have higher sales because of better access to customers.
7.  **Customer Behavior:** Stores keeping the right set of products to meet the local needs of customers will have higher sales.
8.  **Ambiance:** Stores which are well-maintained and managed by polite and humble people are expected to have higher footfall and thus higher sales.
9.  **Season:** Store should sell more after customer's pay day: after 15th or 30th of the month

**Product Level Hypotheses:**

1.  **Brand:** Branded products should have higher sales because of higher trust in the customer.
2.  **Packaging:** Products with good packaging can attract customers and sell more.
3.  **Utility:** Daily use products should have a higher tendency to sell as compared to the specific use products.
4.  **Display Area:** Products which are given bigger shelves in the store are likely to catch attention first and sell more.
5.  **Visibility in Store:** The location of product in a store will impact sales. Ones which are right at entrance will catch the eye of customer first rather than the ones in back.
6.  **Advertising:** Better advertising of products in the store will should higher sales in most cases.
7.  **Promotional Offers:** Products accompanied with attractive offers and discounts will sell more.


Lets move on to the data exploration where we will have a look at the data in detail.

## 2\. Data Exploration

I’ll be performing some basic data exploration here and come up with some inferences about the data.

The first step is to look at the data and try to identify the information which we hypothesized vs the available data. A comparison between the data dictionary on the competition page and out hypotheses is shown below:

![Image of Variables vs Hypothesis](files/../input-data/Variables_vs_Hyphotesis.png)

We can summarize the findings as:

** 9 Features Hypothesized but not found in actual data. **

** 5 Features Hypothesized as well as present in the data **

** 3 Features present in the data but not hypothesized. **


We find features which we hypothesized, but data doesn’t carry and vice versa. We should look for open source data to fill the gaps if possible. Let’s start by loading the required libraries and data. 

In [152]:
import pandas as pd
import numpy as np
import time
import csv
import boto # to download from AWS S3 buckets
import pickle
import warnings
warnings.filterwarnings("ignore")

_start_time = time.time()

# define a easy timing function to use going forward
def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))
    
# utility function- display large dataframes in an html iframe
def df_display(df, lines=500):
    txt = ("<iframe " +
           "srcdoc='" + df.head(lines).to_html() + "' " +
           "width=1000 height=500>" +
           "</iframe>")

    return IPython.display.HTML(txt)


In [153]:
#Let's define variables that will define the behaviour of the whole script
s3_path = 'http://bbts-kaggle.s3.amazonaws.com/bimbo/Pablo/'
use_validation=True # splits train data into train + val sets
val_week_threshold = 9 # (possible values 8 or 9)  - weeks 3,4,5,6,7 are train, and week 8.9 are val
trimmed = False # removes weeks which doesn't have all the lags. If False, fills empty lags with 0
lag = 5  # shifted mean_demand up to "lag" weeks
if (val_week_threshold == 8): lag = 4
retrieve_saved_work = True # skips long jobs by retrieving already saved picked files after Feature 1

In [154]:
#Read files:
tic()
train = pd.read_csv(s3_path +'train.csv',
                           dtype  = {'Semana': 'int8',
                                     'Producto_ID':'int32',
                                     'Cliente_ID':'int32',
                                     'Agencia_ID':'uint16',
                                     'Canal_ID':'int8',
                                     'Ruta_SAK':'int32',
                                     'Venta_hoy':'float32',
                                     'Venta_uni_hoy': 'int8',
                                     'Dev_uni_proxima':'int8',
                                     'Dev_proxima':'float32',
                                     'Demanda_uni_equil':'int32'})
test = pd.read_csv(s3_path +'test.csv',
                           dtype  = {'Semana': 'int8',
                                     'Producto_ID':'int32',
                                     'Cliente_ID':'int32',
                                     'Agencia_ID':'uint16',
                                     'Canal_ID':'int8',
                                     'Ruta_SAK':'int32'})
tac()

MemoryError: 

In [ ]:
# remove unnecessary fields in training data
train.drop(['Venta_uni_hoy', 'Venta_hoy','Dev_uni_proxima', 'Dev_proxima'], axis=1, inplace=True)

In [ ]:
#Since test dataframe is not the same as train dataframe, we make them equal by removing and adding columns
train.insert(0, 'id', np.nan)
test.insert(7, 'Demanda_uni_equil', np.nan)

Now, probably one of the most important steps is to split the train set in train and validation. And never touch the validation set for any feature engineering, and to treat it just like the unseen test set

In [ ]:
if (use_validation):
    tic()
    val = train[train.Semana >= val_week_threshold] # Weeks 8,9
    train = train[train.Semana < val_week_threshold] # Weeks 3,4,5,6,7
    tac()
    

It is a good idea to combine both train and test data sets into one, perform feature engineering and then divide them later again. This saves the trouble of performing the same steps twice on test and train. Lets combine them into a dataframe ‘data’ with a ‘source’ column specifying where each observation belongs.

In [ ]:
tic()
train['source']='train'
if (use_validation): 
    val['source']='val'
test['source']='test'

if (use_validation): 
    data = pd.concat([train,val,test],ignore_index=True)
    print (train.shape, val.shape, test.shape, data.shape)
else:
    data = pd.concat([train,test],ignore_index=True)
    print (train.shape, test.shape, data.shape)
tac()


In [ ]:
#We must now remove the target column=Demanda_uni_equil from the val dataset and keep it away
data['ix'] = data.index # assign and id column to later at the end map the val_targets back to the dataframe
if (use_validation):
    val_target = data[['ix','Demanda_uni_equil']].loc[data['source']=='val'] #save the val targets on a different dataframe
    data.loc[data['source']=='val','Demanda_uni_equil'] =  np.nan # replaces val targets with nan for feature engineering

Thus we can see that data has same #columns but rows equivalent to both test and train. Lets start by checking which columns contain missing values. (takes aprox 30 mins to run!)

In [ ]:
#data.apply(lambda x: sum(x.isnull()))

There doesn't seem to be any missing values (other than the NaN we set on the test and train sets).

Lets look at some basic statistics for numerical variables.

In [ ]:
#data.describe()

Some observations:

   Looking at Demanda_uni_equil (our target), or the amount of product sold per week, we find interesting things:
   
   **1)** The average is 7.22, so in average there is 7 units per week per store sold.
   
   **2)** Looking at the max of 5000, it looks very far fro the mean (3 orders of magnitude), so we must check for an outlier here or a store that is crazy different from the rest.
   
   **3)** Same behaviour we find on Dev_uni_proxima, Venta_hoy and Venta_uni_hoy
   
Looking at the nice data analysis made in R by Faviens, here: https://www.kaggle.com/fabienvs/grupo-bimbo-inventory-demand/notebook-8a62eda039a3b0b944cf/notebook we corroborate the outlier(s):
There is a massive client: Puebla Remision
   
![Image of size of Customers]( https://www.kaggle.io/svf/267812/783a24d1dd546819a44914f996b249e8/__results___files/figure-html/unnamed-chunk-16-1.png)
   

Moving to nominal (categorical) variable, lets have a look at the number of unique values in each of them.

In [ ]:
#data.apply(lambda x: len(x.unique()))

So, in train and test sets, we have 552 Agencies(depots), 890k clients (we might have some repeated clients due to typos when enterind data), 1833 products (we might have some repeated products here based on typos) and 3620 routes

In [ ]:
# Let's see how many records we have per week
for i in range(3,12):
    print("Semana"+repr(i)+" =\t" + repr(data[data["Semana"]==i].Semana.count()))

As stated in the Kaggle competition - Week 10 and 11 is sampled down aprox 70%. According to Kaggle, this was done so the scoring of candidates didn't take extremely long.

## 3\. Data Cleaning

This step involves imputing missing values and treating outliers. As we saw before, there are no missing values. Regarding outliers, there seem to be an obvious one, but we are going to see later on if its necessary to treat it differently.

My initial reaction would be to see if anything with the word REMISION is on the test set. if not, then delete it. See this discussion: https://www.kaggle.com/c/grupo-bimbo-inventory-demand/forums/t/22037/puebla-remission/126053

In [ ]:
#Let's find out who are the clients with the word REMISION on it
client_name = pd.read_csv('./input-data/cliente_tabla.csv')
cliente_id_name_train = pd.merge(train,client_name, on='Cliente_ID')
cliente_id_name_test = pd.merge(test,client_name, on='Cliente_ID')

In [ ]:
cliente_id_name_train.head()

In [ ]:
cliente_id_name_train[cliente_id_name_train.NombreCliente.str.contains('REMISION')].count()

As we can see above, the word "REMISION" shows up 140k times on the train set. Let's see the test set:

In [ ]:
cliente_id_name_test[cliente_id_name_test.NombreCliente.str.contains('REMISION')].count()

12k rows shows up the word REMISION on the test set. This implies that it has to be predicted as well. We cannot eliminate it.

## 4\. Feature Engineering

We explored some nuances in the data in the data exploration section. Lets move on to resolving them and making our data ready for analysis. We will also create some new variables using the existing ones in this section.

In [ ]:
#First thing we need to do is to transform our target ( Demanda_uni_equil) to log(1 + demand) - this makes sense since we're 
#trying to minimize rmsle vs the mean which minimizes rmse. At the end of the modeling (for submission) we need to reverse it 
#by applying expm1(x)
data['log_target'] = np.log1p(data["Demanda_uni_equil"])

In [ ]:
data.head()

In [ ]:
#Let's also create all the grouping dataframes we are going to need 
tic()

global_mean = data['log_target'].mean()
prod_mean = data.groupby('Producto_ID').agg({'log_target': 'mean' })
client_mean = data.groupby('Cliente_ID').agg({'log_target': 'mean' })
prod_client_mean = data.groupby(['Producto_ID', 'Cliente_ID']).agg({'log_target': 'mean' })
semana_client_prod_mean = data.groupby(['Semana','Cliente_ID','Producto_ID']).agg({'log_target': 'mean'})
ruta_cliente_prod_mean = data.groupby(['Ruta_SAK','Cliente_ID','Producto_ID']).agg({'log_target': 'mean'})

tac()

###  Feature 1:  Mean of pairs cliente-producto

First and foremost, we want to create the average (mean) demand per each pair of cliente-producto, if we don't have the demand of the pair,  we calculate the average demand of the product, and if we don't have the demand of the product(new introduced product that week), then we calculate the average demand of the client, and if we don't have any (new client and  on that week), we default to the global

In [ ]:
tic()
prod_mean_dict = prod_mean.to_dict()
client_mean_dict = client_mean.to_dict()
prod_client_mean_dict = prod_client_mean.to_dict()
tac()

In [ ]:
def gen_pairs_mean_feature(key):
    key = tuple(key)
    product = key[0]
    client = key[1]
    
    val = prod_client_mean_dict['log_target'][(product,client)]
    if np.isnan(val):
        val = prod_mean_dict['log_target'][(product)]
        if np.isnan(val):
            val = client_mean_dict['log_target'][(client)]
            if np.isnan(val):
                val = global_mean
            
    return val

In [ ]:
if not (retrieve_saved_work):
    tic()
    data['pairs_mean'] = data[['Producto_ID', 'Cliente_ID']].apply(lambda x:gen_pairs_mean_feature(x), axis=1)
    tac()

In [ ]:
data.head()

In [ ]:
#Saving the work
if not (retrieve_saved_work):
    tic()
    output = open('./input-data/Data_pairs_mean.pkl', 'wb')
    pickle.dump(data, output)
    tac()

### Feature 2: Lags - Demand per client-product pair for prior weeks
Based on this blog: http://blog.nycdatascience.com/student-works/predicting-demand-historical-sales-data-grupo-bimbo-kaggle-competition/

As this script said: https://www.kaggle.com/bpavlyshenko/grupo-bimbo-inventory-demand/bimbo-xgboost-r-script-lb-0-457/code
It is important to know what were the previous weeks sales. If the previous week, too many products were supplied and they were not sold, the next week this product amount, supplied to the same store, will be decreased. So it is very important to included lag values of target variable as a feature to predict the next sales.

In [ ]:
#Retrieve the saved work
if (retrieve_saved_work):
    tic()
    pkl_file = open('./input-data/Data_pairs_mean.pkl', 'rb')
    data = pickle.load(pkl_file)
    tac()

In [ ]:
df = semana_client_prod_mean.reset_index() # we convert the index to columns for later use

In [ ]:
df.head()

In [ ]:
# Let's see how many records we have per week on the semana_cliente_Producto groups vs the raw dataset
for i in range(3,12):
    print("Semana"+repr(i)+" =\t" + repr(data[data["Semana"]==i].Semana.count())+ " (raw)\t" +
            repr(df[df["Semana"]==i].Semana.count()) + " (group)\t" +  
            repr(data[data["Semana"]==i].Semana.count() - df[df["Semana"]==i].Semana.count()) + " (diff)"
         )

As we can see above, there are repeated combinations of client-product on each week.

In [ ]:
#Before we start adding lags and removing rows, let's see the size of our dataset
size_data = data.memory_usage().sum()
print(size_data)

In [ ]:
#here we add the number of lags we want
tic()

for i in range(1,lag+1):
    df['Semana'] += 1
    df.rename(columns={df.columns[3]: 'Log_Target_mean_lag%d' %(i)}, inplace=True)
    data = pd.merge(data,df, how = 'left', on = ['Semana','Cliente_ID','Producto_ID']) #here we add the lag to the dataset
    data['Log_Target_mean_lag%d' %(i)].fillna(0, inplace=True) # we replace the client-product log mean NaN/Not found on the week before with ZERO
    if(trimmed): data = data[data.Semana != i+2] # here we delete the week rows we dont have lags for
   
tac()

In [ ]:
#Let's see how many NaN or Nulls we have
#data.apply(lambda x: sum(x.isnull()))

The above looks correct! the only NaN shown are the variables that are not avaibable on the val or test set, everyting else looks good


In [ ]:
# Let's see how many records we have per week and make sure we didn't delete any data from our important weeks
for i in range(3,12):
    print("Semana"+repr(i)+" =\t" + repr(data[data["Semana"]==i].Semana.count()))

The above looks correct! we deleted the week rows we don't have lags for, and we kept the original amount of rows for the remaining weeks.

In [ ]:
#Now let's see how much was the data set size reduced/increased
new_size_data = data.memory_usage().sum()
print("Dataset size changed in " + repr((new_size_data - size_data )*100/new_size_data) + "%")

Good! the data set was greatly reduced (this means faster modeling)

In [ ]:
data.head()

###  Feature 4:  Calculates de sum of prior weeks Log mean Demands

In [ ]:
#We want to sum the lags up until week 9, this means that we need to sum lag2 and up.
data['Lags_sum'] = 0
for i in range(1,lag+1):
    data['Lags_sum'] += data['Log_Target_mean_lag%d' %(i)]

In [ ]:
data.head()

### Feature 5: Create a broad category of Brand of item (brand hypothesis)
Let's preprocess products a little bit. I borrowed some of the preprocessing from here: https://www.kaggle.com/vykhand/grupo-bimbo-inventory-demand/exploring-products

In [ ]:
products  =  pd.read_csv("input-data/producto_tabla.csv")
products  =  pd.read_csv("input-data/producto_tabla.csv")
#products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)#python 2.7
products['short_name'] = products.NombreProducto.str.extract('^(\D*)')#python 3.0
#products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$')
#w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)')
products['weight'] = w[0].astype('float')*w[1].map({'Kg':1000, 'g':1})
#products['pieces'] =  products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')
products['pieces'] =  products.NombreProducto.str.extract('(\d+)p ').astype('float')
products.head()

In [ ]:
products.tail()

In [ ]:
products.brand.value_counts()

In [ ]:
products.brand.nunique()

In [ ]:
products_id_brand  = products[['Producto_ID', 'brand']].copy()

In [ ]:
data = pd.merge(data, products_id_brand, on='Producto_ID')

In [ ]:
data.head()

### Feature 6: Create clusters of Products (utility hypothesis) - ramdonly pick 30 clusters

In [ ]:
#Read files:
product_clusters = pd.read_csv('input-data/producto_clusters.csv')

In [ ]:
product_clusters.tail()

In [ ]:
print (product_clusters["cluster"].value_counts())

In [ ]:
products_id_clusters = product_clusters[['Producto_ID', 'cluster']].copy()
products_id_clusters.rename(columns={'cluster': 'prodtype_cluster'}, inplace=True)

In [ ]:
products_id_clusters.tail()

In [ ]:
data = pd.merge(data, products_id_clusters, on='Producto_ID')

In [ ]:
data.head()

### Feature 7: Create a category of Size of store based on Number of Agencies and Routes and Sales Channels that serve the store

In [ ]:
#Determine pivot table
Rutas_per_store = data.pivot_table(values=["Ruta_SAK"], index=["Cliente_ID"], aggfunc=pd.Series.nunique)

In [ ]:
Rutas_per_store.describe()

In [ ]:
#Agencies_per_store = data.pivot_table(values=["Agencia_ID"], index=["Cliente_ID"], aggfunc=pd.Series.nunique)

In [ ]:
#Agencies_per_store.describe()

In [ ]:
#Canals_per_store = data.pivot_table(values=["Canal_ID"], index=["Cliente_ID"], aggfunc=pd.Series.nunique)

In [ ]:
#Canals_per_store.describe()

It doesn't look that we can Bin on Canal_ID or Agencia_ID since they are not at least semi evenly ditributed, but it does look like Ruta_SAK is a good option based on the percentiles distribution"

In [ ]:
Rutas_per_store.rename(columns={'Ruta_SAK': 'Qty_Ruta_SAK'}, inplace=True)

In [ ]:
#Mergin Routa_Sak's per client to data df
data = pd.merge(data,Rutas_per_store,right_index=True, left_on='Cliente_ID')

In [ ]:
data.tail()

In [ ]:
#Binning:
def binning(col, cut_points, labels=None):
  #Define min and max values:
  minval = col.min()
  maxval = col.max()

  #create list by adding min and max to cut_points
  break_points = [minval] + cut_points + [maxval]

  #if no labels provided, use default labels 0 ... (n-1)
  if not labels:
    labels = range(len(cut_points)+1)

  #Binning using cut function of pandas
  colBin = pd.cut(col,bins=break_points,labels=labels,include_lowest=True)
  return colBin

#Binning Qty_Ruta_SAK:
cut_points = [2,4,10]
labels = ["low","medium","high","very high"]
data["Qty_Ruta_SAK_Bin"] = binning(data["Qty_Ruta_SAK"], cut_points, labels)
print (pd.value_counts(data["Qty_Ruta_SAK_Bin"], sort=False))

In [ ]:
#We don't need Qty_Ruta_Sak anymore
data.drop(['Qty_Ruta_SAK'],axis=1,inplace=True)

In [ ]:
data.head()

### Feature 8: Create a category of location based on zip code (embedded on town table)

In [ ]:
import re 
import os
import time
towns = pd.read_csv("input-data/town_state.csv")
L = lambda x: list(map(int, re.findall('\d+', x)))[0]
towns['ZipCode'] = towns.Town.apply(L) 
towns['ZipCode'] = np.uint16(towns['ZipCode'])

In [ ]:
zipcodes_df = towns[['Agencia_ID', 'ZipCode']].copy()

In [ ]:
zipcodes_df.head()

In [ ]:
data = pd.merge(data, zipcodes_df, on='Agencia_ID')

In [ ]:
data.head()

### Feature 9: Week of the month counter

The idea is to have an indicator of what week of the month the current data belongs. This is to see if there is a monthly pattern that the algorithm can pick up.

In [ ]:
data['week_ct'] = data['Semana'].apply(lambda x: x%4)

In [ ]:
data.tail()

### Feature 10: Clusters

#### 10.1 Client type clusters
Thanks to AbderRahman Sobh - https://www.kaggle.com/abbysobh/grupo-bimbo-inventory-demand/classifying-client-type-using-client-names/comments
for doing the great code

In [ ]:
client_types = pd.read_csv('./input-data/client_types.csv',header=0)

In [ ]:
client_types.head()

In [ ]:
data = data.merge(client_types.drop_duplicates(subset=['Cliente_ID']), how="left")

In [ ]:
data.tail()

#### 10.2  Clusters based on demand
We are going to load a previous calculate tables of Agencia,Ruta, Producto and Cliente Clusters based on demand mean and std

In [ ]:
import h2o
import imp
from h2o.estimators.kmeans import H2OKMeansEstimator

In [ ]:
# Start a local instance of the H2O engine.
h2o.init();

In [ ]:
from sklearn.cluster import KMeans
import math as math
from matplotlib import pyplot as plt
%matplotlib inline

#### Let's try to find the right amount of clusters

There are three diagnostics that will be used to help with determining the number of clusters: total within cluster sum of squares, AIC, and BIC.

Total within cluster sum of squares measures sums the distance from each point in a cluster to that point's assigned cluster center. This is the minimization criteria of kmeans. The standard guideline for picking the number of clusters is to look for a 'knee' in the plot, showing where the total within sum of squares stops decreasing rapidly. Total within cluster sum of squares can be difficult to intepret, with the criteria being to look for an arbitrary knee in the plot.

With this challenge from total within cluster sum of squares, we will also use two merit statistics for determining the number of clusters. AIC and BIC are both measures of the relative quality of a statistical model. AIC and BIC introduce penality terms for the number of parameters in the model to counter the problem of overfitting; BIC has a larger penality term than AIC. With these merit statistics one is to look for the number of clusters that minimize the statistic.

Here we build a method for extracting the inputs for each diagnostics and calculating the AIC and BIC values on a model. Each model is then inspected by the method and the results plotted for quick analysis.


In [ ]:
def diagnostics_from_clusteringmodel(model):
    total_within_sumofsquares = model.tot_withinss()
    number_of_clusters = len(model.centers())
    number_of_dimensions = len(model.centers()[0])
    number_of_rows = sum(model.size())
    
    aic = total_within_sumofsquares + 2 * number_of_dimensions * number_of_clusters
    bic = total_within_sumofsquares + math.log(number_of_rows) * number_of_dimensions * number_of_clusters
    
    return {'Clusters':number_of_clusters,
            'Total Within SS':total_within_sumofsquares, 
            'AIC':aic, 
            'BIC':bic}

In [ ]:
def cluster_and_plot(feat_name, k_range):

    global data
    
    # run clustering by demand using all data set and our pairs_mean feature
    feat = data.loc[:,feat_name]
    tar = data.loc[:,'pairs_mean']
    feat_tar = pd.concat([feat, tar], axis=1)
        
    # group by feature
    grouped_ft = feat_tar.groupby(feat_name)
    grouped_ft = pd.merge(grouped_ft.median().reset_index(), grouped_ft.std().reset_index(), how='left', on=[feat_name])
    
    demand_info_unique = grouped_ft.iloc[:,1:]
    demand_info_unique[pd.isnull(demand_info_unique)] = 0
        
    # we use the kmeans clustering algorithm
    h2odf = h2o.H2OFrame(demand_info_unique)
    results = [H2OKMeansEstimator(k=clusters, init="Random", seed=2, standardize=True) for clusters in k_range]
    for estimator in results:
        estimator.train(x = h2odf.col_names, training_frame = h2odf)

    diagnostics = pd.DataFrame( [diagnostics_from_clusteringmodel(model) for model in results])
    diagnostics.set_index('Clusters', inplace=True)
    diagnostics.plot(kind='line');


In [ ]:
agencia_k_range = range(10, 50, 5)
tic()
cluster_and_plot('Agencia_ID', agencia_k_range)
tac()

It Looks Like Clusters = 20 is a good choice

In [ ]:
ruta_k_range = range(60, 300, 30)
tic()
cluster_and_plot('Ruta_SAK', ruta_k_range)
tac()

It Looks Like Clusters = 80 is a good choice

In [ ]:
producto_k_range = range(30, 150, 15)
tic()
cluster_and_plot('Producto_ID', producto_k_range)
tac()

It Looks Like Clusters = 45 is a good choice

In [ ]:
tic()
cliente_k_range = [10,100,300,500]
#cluster_and_plot('Cliente_ID', cliente_k_range)
tac()

It looks like 300 or 400 would work

#### Now we the right amount of centroids, we process the clustering and fitting

In [ ]:
# --- HYPERPARAMETERS FOR K-MEANS ---

# number of clusters to group depot/route/produc (if 0 will not be added as feature)
num_clusters_agencia = 20
num_clusters_ruta = 80
num_clusters_producto = 45
num_clusters_cliente = 300 # Amount of centroids for k-means client clustering by demand

In [ ]:
def cluster_and_save(feat_name, num_clusters):
    '''
    Input: 
        - idx: the index of the feature we want to cluster, one-hot-encode, and add to our features
        - num_clusters: the number of clusters we want to use to group the feature values
    '''   
    global data

    # run clustering by demand using info from week 3-9
    feat = data.loc[:, feat_name]
    tar = data.loc[:, 'pairs_mean']
    feat_tar = pd.concat([feat, tar], axis=1)
        
    # group by feature
    grouped_ft = feat_tar.groupby(feat_name)
    grouped_ft = pd.merge(grouped_ft.median().reset_index(), grouped_ft.std().reset_index(), how='left', on=[feat_name])
    
    demand_info_unique = grouped_ft.iloc[:,1:]
    demand_info_unique[pd.isnull(demand_info_unique)] = 0
    
    # we use the kmeans clustering algorithm
    h2odf = h2o.H2OFrame(demand_info_unique)
    kmeans = H2OKMeansEstimator(k=num_clusters, init="Random", seed=2, standardize=True)

    kmeans.train(x = h2odf.col_names, training_frame = h2odf)
    clusters_h2odf = kmeans.predict(h2odf)
    clustersdf = clusters_h2odf.as_data_frame(True)
    clusters = clustersdf.predict.get_values()
    
    # plot demand/cluster
    plt.figure(figsize=(15, 3))
    for c in range(num_clusters):
        d_median = demand_info_unique.iloc[clusters==c,0]
        d_std = demand_info_unique.iloc[clusters==c,1]
        plt.plot(d_median,d_std,'.')
        plt.xlabel('median')
        plt.ylabel('std')
    plt.savefig("./input-data/h2o-clustByDem_{}_{}".format(feat_name, num_clusters))
    
    # create new dataframe to save the mapping from feature ID to cluster ID
    feat_clust_map = pd.DataFrame(data = grouped_ft.iloc[:,0], columns=[feat_name])
    feat_clust_map.insert(1, feat_name+'_clust_ID', clusters)

    # save the new feature in files for others to use 
    feat_clust_map.to_csv("./input-data/h2o-clustByDem_{}.csv".format(feat_name), index=False)

In [ ]:
tic()
print("Agencia Clusters")
cluster_and_save('Agencia_ID', num_clusters_agencia)
tac()

In [ ]:
tic()
print("Ruta Clusters")
cluster_and_save('Ruta_SAK', num_clusters_ruta)
tac()

In [ ]:
tic()
print("Producto Clusters")
cluster_and_save('Producto_ID', num_clusters_producto)
tac()

In [ ]:
tic()
print("Cliente Clusters")
cluster_and_save('Cliente_ID', num_clusters_cliente)
tac()

In [ ]:
#Read files:
agencia_by_dem_clust = pd.read_csv('input-data/h2o-clustByDem_Agencia_ID.csv')
ruta_by_dem_clust = pd.read_csv('input-data/h2o-clustByDem_Ruta_SAK.csv')
prod_by_dem_clust = pd.read_csv('input-data/h2o-clustByDem_Producto_ID.csv')
cliente_by_dem_clust = pd.read_csv('input-data/h2o-clustByDem_Cliente_ID.csv')

In [ ]:
data = pd.merge(data, prod_by_dem_clust, on='Producto_ID')
data = pd.merge(data, ruta_by_dem_clust, on='Ruta_SAK')
data = pd.merge(data, agencia_by_dem_clust, on='Agencia_ID')
data = pd.merge(data, cliente_by_dem_clust, on='Cliente_ID')

In [ ]:
data.tail()

## Encode nominal features, one-hot Categorical features and Scale numerical features
Since scikit-learn accepts only numerical variables, so i have to convert all categories of nominal variables into numeric types.

Lets start with coding all low cardinality object/nominal categorical variables (brand, Qty_Ruta_SAK_Bin, NombreCliente)  as numeric using ‘LabelEncoder’ from sklearn’s preprocessing module.

In [ ]:
print (data.dtypes)

In [ ]:
#data.apply(lambda x: len(x.unique()))

In [ ]:
#Here we convert to numbers the nominal variables
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

var_mod = ['brand', 'Qty_Ruta_SAK_Bin', 'NombreCliente']
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [ ]:
#data.describe()

One-Hot-Coding refers to creating dummy variables, one for each category of a categorical variable. For example, the 'cluster' variable has 30 categories. One hot coding will remove this variable and generate 30 new variables. Each will have binary numbers – 0 (if the category is not present) and 1(if category is present).
Categorical variables are intentionally (for censorship) or implicitly encoded as numerical variables in order to be used as features in any given model.

e.g. [house, car, tooth, car] becomes [0,1,2,1].

This imparts an ordinal property to the variable, i.e. house < car < tooth.

As this is ordinal characteristic is usually not desired, one hot encoding is necessary for the proper representation of the distinct elements of the variable.

-- This can be done using ‘get_dummies’ function of Pandas.


In [ ]:
#One Hot Coding of the categorical variables
onehot_categoricals = False # We don't need to do this now. It is better to do it in the modeling part, and use RAM not disk space
if (onehot_categoricals):
    tic()
    data = pd.get_dummies(data, columns=['week_ct','prodtype_cluster','Qty_Ruta_SAK_Bin', 'brand', 'NombreCliente','ZipCode'])
    tac()

In [ ]:
#Scaling of the numerical values
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

s_cols = ['pairs_mean', 'Lags_sum'] # s_cols is an array of all numerical features
for i in range(1,lag):
    s_cols.insert(0,'Log_Target_mean_lag{}'.format(i))

data[s_cols] = min_max_scaler.fit_transform(data[s_cols])

## 5\. Exporting Data

In [ ]:
#Divide into test and train:

tic()
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]
if (use_validation):
    val = data.loc[data['source']=="val"]
    val.drop('Demanda_uni_equil',axis=1,inplace=True) # here all Demanda_uni_equil = nan
    # we merge back the val_targets to the val dataset
    val = pd.merge(val,val_target, on='ix') # we add the true Demanda_uni_equil back
    val['log_target'] = np.log1p(val["Demanda_uni_equil"]) # adds the log_target col

#Drop unnecessary columns: note - we are dropping Demanda_uni_equil since we replaced it by log_target
train.drop(['id','Demanda_uni_equil','source','ix'],axis=1,inplace=True)
test.drop(['Demanda_uni_equil','source','ix'],axis=1,inplace=True)
if (use_validation):
    val.drop(['id','Demanda_uni_equil','source','ix'],axis=1,inplace=True)

#Export files as modified versions:
sufix=""
if (use_validation): 
    sufix += "_holdout"
    sufix += repr(val_week_threshold)
if (trimmed): sufix += "_trimmed"

print("Writing: train_modified"+sufix+".csv  to disk ...")
train.to_csv("./input-data/train_modified"+sufix+".csv", index=False, quoting=csv.QUOTE_NONE)
print("Writing: test_modified"+sufix+".csv  to disk ...")
test.to_csv("./input-data/test_modified"+sufix+".csv", index=False, quoting=csv.QUOTE_NONE)
if (use_validation):
    print("Writing: val_modified"+sufix+".csv  to disk ...")
    val.to_csv("./input-data/val_modified"+sufix+".csv", index=False, quoting=csv.QUOTE_NONE)

tac()

In [ ]:
train.tail()